In [8]:
# Ordinal XGBoost, Threshold models, Three severity levels


import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from scipy.special import expit
from xgboost import XGBRegressor
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    average_precision_score,
)


from pathlib import Path
BASE_DIR = Path().resolve()

CSV_PATH = BASE_DIR / "storms_data.csv"
storms_data = pd.read_csv(CSV_PATH)

In [9]:
#Column configuration (MATCH YOUR CSV)
OUTAGE_COL = "max_outage_after_24h"
BASELINE_COL = "baseline_outage_median"
HU_COL = "housing_units"


# Feature configuration (edit if needed)
feature_cols = [
    "era_i10fg_max_total_48h",
    "era_tp_max_total_48h",
    "era_crr_max_total_48h",
    'housing_units_by_area',
    "overhead_circuits",
    "n_points",
    "n_urban",
    "season_code",
    "cbp_emp_total",
]

# -------------------------
# 3) Sanity checks: required columns exist
# -------------------------
required_cols = list(set(feature_cols + [OUTAGE_COL, BASELINE_COL, HU_COL]))
missing = [c for c in required_cols if c not in storms_data.columns]
if missing:
    raise KeyError(f"Missing required columns in CSV: {missing}")

# -------------------------
# 4) Config
# -------------------------
RANDOM_STATE = 42
TEST_SIZE = 0.2
USE_EXCESS = True  # True: (max_outage - baseline), False: use max_outage

# Engineering thresholds (4-level specification)
# Level 0: <0.5% ; Level 1: [0.5%,2%) ; Level 2: [2%,5%) ; Level 3: >=5%
CUTS = [0.005, 0.02, 0.05]

# XGBoost parameters (safe defaults)
XGB_PARAMS = dict(
    objective="binary:logistic",
    eval_metric="logloss",
    n_estimators=800,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.0,
    reg_lambda=1.0,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

# -------------------------
# 5) Build sev_ratio and ordinal labels
# -------------------------
df0 = storms_data.copy()

max_out = pd.to_numeric(df0[OUTAGE_COL], errors="coerce")
base = pd.to_numeric(df0[BASELINE_COL], errors="coerce")
hu = pd.to_numeric(df0[HU_COL], errors="coerce").replace(0, np.nan)

z = (max_out - base) if USE_EXCESS else max_out
df0["sev_ratio"] = (z / hu).clip(lower=0)  # truncate negative noise to 0

# Convert features to numeric where possible (keep NaN if non-parsable)
for c in feature_cols:
    df0[c] = pd.to_numeric(df0[c], errors="coerce")

df = df0.dropna(subset=feature_cols + ["sev_ratio"]).copy()

print("\n[Info] sev_ratio distribution:")
print(df["sev_ratio"].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99]))

def ratio_to_level_4(x: float, cuts=CUTS) -> int:
    if x < cuts[0]:
        return 0
    elif x < cuts[1]:
        return 1
    elif x < cuts[2]:
        return 2
    else:
        return 3

df["y4"] = df["sev_ratio"].apply(ratio_to_level_4).astype(int)

print("\n[Check] y4 class proportions (%):")
print((df["y4"].value_counts(normalize=True).sort_index().mul(100).round(2)).to_string())

if df["y4"].nunique() < 2:
    raise ValueError("y4 has <2 classes under current CUTS; adjust thresholds.")

# ---- Merge Level 1 and Level 2 ----
# original: 0,1,2,3  -> merged: 0,1,2 where (1,2)->1 and 3->2
df["y"] = df["y4"].replace({2: 1, 3: 2}).astype(int)

print("\n[Check] merged y (target) class proportions (%):")
print((df["y"].value_counts(normalize=True).sort_index().mul(100).round(2)).to_string())

levels = sorted(df["y"].unique().tolist())
if levels != [0, 1, 2]:
    raise ValueError(f"Expected merged y to have levels [0,1,2], got {levels}")

# Train/test split (shared for all thresholds)

X = df[feature_cols].copy()
y = df["y"].astype(int).copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y,
)

# For K=3 ordinal levels, train 2 cumulative models: P(y>=1), P(y>=2)
y_ge_1_train = (y_train >= 1).astype(int)
y_ge_2_train = (y_train >= 2).astype(int)

y_ge_1_test = (y_test >= 1).astype(int)
y_ge_2_test = (y_test >= 2).astype(int)

print("\n[Check] cumulative rates:")
print(f"P(y>=1) train={y_ge_1_train.mean():.3f}, test={y_ge_1_test.mean():.3f}")
print(f"P(y>=2) train={y_ge_2_train.mean():.3f}, test={y_ge_2_test.mean():.3f}")

# Train cumulative binary XGB models

m_ge_1 = xgb.XGBClassifier(**XGB_PARAMS)
m_ge_2 = xgb.XGBClassifier(**XGB_PARAMS)

m_ge_1.fit(X_train, y_ge_1_train)
m_ge_2.fit(X_train, y_ge_2_train)

# Predict and enforce monotonicity

p_ge_1 = m_ge_1.predict_proba(X_test)[:, 1]
p_ge_2 = m_ge_2.predict_proba(X_test)[:, 1]

# Enforce monotonicity: P(y>=1) >= P(y>=2)
p_ge_2 = np.minimum(p_ge_2, p_ge_1)

# Convert to 3-class probabilities:
P0 = 1.0 - p_ge_1
P1 = p_ge_1 - p_ge_2
P2 = p_ge_2

proba_ord = np.vstack([P0, P1, P2]).T  # (n,3)
y_pred = np.argmax(proba_ord, axis=1)

#Reports

print(classification_report(y_test, y_pred, digits=4))
print(confusion_matrix(y_test, y_pred))

try:
    print(f"AUC P(y>=1): {roc_auc_score(y_ge_1_test, p_ge_1):.4f}")
except ValueError as e:
    print("AUC P(y>=1) not available (likely only one class in y_ge_1_test).", e)

try:
    print(f"AUC P(y>=2): {roc_auc_score(y_ge_2_test, p_ge_2):.4f}")
except ValueError as e:
    print("AUC P(y>=2) not available (likely only one class in y_ge_2_test).", e)


# Feature importance (per-threshold + average)
imp1 = pd.Series(m_ge_1.feature_importances_, index=feature_cols, name="P(y>=1)")
imp2 = pd.Series(m_ge_2.feature_importances_, index=feature_cols, name="P(y>=2)")

imp_table = pd.concat([imp1, imp2], axis=1)
imp_avg = imp_table.mean(axis=1).sort_values(ascending=False)

print("\n===== Feature Importance (Average over thresholds) =====\n")
print(imp_avg.to_string())

print("\n===== Feature Importance (Per threshold) =====\n")
print(imp_table.sort_values(by="P(y>=2)", ascending=False).to_string())


[Info] sev_ratio distribution:
count    9667.000000
mean        0.020446
std         0.056287
min         0.000000
50%         0.006146
75%         0.015572
90%         0.047092
95%         0.087721
99%         0.250419
max         0.997414
Name: sev_ratio, dtype: float64

[Check] y4 class proportions (%):
y4
0    44.01
1    36.03
2    10.47
3     9.50

[Check] merged y (target) class proportions (%):
y
0    44.01
1    46.50
2     9.50

[Check] cumulative rates:
P(y>=1) train=0.560, test=0.560
P(y>=2) train=0.095, test=0.095
              precision    recall  f1-score   support

           0     0.9438    0.9471    0.9455       851
           1     0.9509    0.9488    0.9499       899
           2     0.9727    0.9674    0.9700       184

    accuracy                         0.9498      1934
   macro avg     0.9558    0.9544    0.9551      1934
weighted avg     0.9499    0.9498    0.9499      1934

[[806  42   3]
 [ 44 853   2]
 [  4   2 178]]
AUC P(y>=1): 0.9912
AUC P(y>=2): 0.9970



In [10]:
threshold = 10

storms_data["y_outage"] = (
    storms_data["max_outage_after_24h"]
    > storms_data["baseline_outage_median"] + threshold
).astype(int)

feature_cols = [
    # Hazard (storm intensity)
    "era_i10fg_max_total_48h",
    "era_tp_max_total_48h",
    "era_crr_max_total_48h",

    # Exposure
    'housing_units_by_area',
    'overhead_circuits',

    # Fragility / resilience
    'n_points', 'n_urban',

    # Context
    "season_code",'cbp_emp_total'
]


df = storms_data.dropna(subset=feature_cols + ["y_outage"]).copy()

X = df[feature_cols]
y = df["y_outage"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

model = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    n_estimators=500,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
)

model.fit(X_train, y_train)

p_test = model.predict_proba(X_test)[:, 1]
y_pred = (p_test >= 0.5).astype(int)   # 或者用 model.predict(X_test)

auc = roc_auc_score(y_test, p_test)
ap = average_precision_score(y_test, p_test)

print(f"AUC = {auc:.3f}")
print(f"AP  = {ap:.3f}")

print("\n===== Classification Report =====\n")
print(classification_report(y_test, y_pred))

print("\n===== Confusion Matrix =====\n")
print(confusion_matrix(y_test, y_pred))

AUC = 0.991
AP  = 0.999

===== Classification Report =====

              precision    recall  f1-score   support

           0       0.98      0.84      0.91       129
           1       0.99      1.00      1.00      2296

    accuracy                           0.99      2425
   macro avg       0.99      0.92      0.95      2425
weighted avg       0.99      0.99      0.99      2425


===== Confusion Matrix =====

[[ 109   20]
 [   2 2294]]
